In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import msgpack
import glob
from tqdm import tqdm
import midi_manipulation
from tensorflow.python.ops import control_flow_ops

In [2]:
path="/home/bhaktipriya/ML/rnn_music/blues"
files=glob.glob('{}/*.*mid*'.format(path))
songs=[]
for f in tqdm(files):
            song = np.array(midi_manipulation.midiToNoteStateMatrix(f))
            if np.array(song).shape[0] > 50:
                songs.append(song)

100%|██████████| 126/126 [00:03<00:00, 37.82it/s]


In [3]:
#MusicParams

lowestnote=midi_manipulation.lowerBound
highestnote=midi_manipulation.upperBound
noterange=highestnote-lowestnote

In [4]:
#Hyperparams
timesteps=15 #no of notes to create at a time
nv=2*noterange*timesteps #visible layer size. 
nh=50 #hidden layer size.
epochs=200 
batch_size=100  
lr=tf.constant(0.005, tf.float32) #learning rate 

In [5]:
#Placeholders & vars for Network

x=tf.placeholder(tf.float32, [None, nv], name="x") #The placeholder for data
W=tf.Variable(tf.random_normal([nv, nh], 0.01), name="W")#weight matrix
bh=tf.Variable(tf.zeros([1, nh],  tf.float32, name="bh")) #bias for hidden layer
bv=tf.Variable(tf.zeros([1, nv],  tf.float32, name="bv")) #bias for visible layer


In [6]:
def sample(probs):
    return tf.floor(probs+tf.random_uniform(tf.shape(probs), 0, 1))

def gibbs_step(count, k, xk):
        #Runs a single gibbs step. The visible values are initialized to xk
        hk = sample(tf.sigmoid(tf.matmul(xk, W) + bh))
        xk = sample(tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv))
        return count+1, k, xk

def gibbs_sample(k):
    #Gibbs sample is used to approximate the distribution of the RBM(defined by W, bh, bv)
    
    #Run gibbs steps for k iterations
    ct = tf.constant(0) #counter
    [_, _, x_sample] = control_flow_ops.while_loop(lambda count, num_iter, *args: count < num_iter,gibbs_step, [ct, tf.constant(k), x], 1, False)                
    #This is not strictly necessary in this implementation, but if you want to adapt this code to use one of TensorFlow's
    #optimizers, you need this in order to stop tensorflow from propagating gradients back through the gibbs step
    x_sample = tf.stop_gradient(x_sample) 
    return x_sample


In [7]:
h=sample(tf.sigmoid(tf.matmul(x, W) + bh)) 

x_sample=gibbs_sample(1) 

h_sample=sample(tf.sigmoid(tf.matmul(x_sample, W) + bh)) 

In [8]:
#Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
size_bt=tf.cast(tf.shape(x)[0], tf.float32)
eta=lr/size_bt
W_upd=tf.mul(eta, tf.sub(tf.matmul(tf.transpose(x), h), tf.matmul(tf.transpose(x_sample), h_sample)))
bv_upd=tf.mul(eta, tf.reduce_sum(tf.sub(x, x_sample), 0, True))
bh_upd=tf.mul(eta, tf.reduce_sum(tf.sub(h, h_sample), 0, True))
updt=[W.assign_add(W_upd), bv.assign_add(bv_upd), bh.assign_add(bh_upd)]

In [9]:
sess=tf.Session()
init=tf.initialize_all_variables()
sess.run(init)
for epoch in tqdm(range(epochs)):
            for song in songs:
                song=np.array(song)
                #reshaping song into chunks of timestep size
                chunks=song.shape[0]/timesteps
                dur=chunks*timesteps
                song=song[:dur]
                song=np.reshape(song, [chunks, song.shape[1]*timesteps])
                #Train the RBM on batch_size examples at a time
                for i in range(1, len(song), batch_size): 
                    tr_x=song[i:i+batch_size]
                    sess.run(updt, feed_dict={x: tr_x})


100%|██████████| 200/200 [02:29<00:00,  1.35it/s]


In [10]:
sample=gibbs_sample(1).eval(session=sess, feed_dict={x: np.zeros((10, nv))})
generatedfiles=[]
for i in range(sample.shape[0]):
            if not any(sample[i,:]):
                continue
            #save the op vector as a midi file
            S = np.reshape(sample[i,:], (timesteps, 2*noterange))
            midi_manipulation.noteStateMatrixToMidi(S, "gen_snippet{}".format(i))
            generatedfiles.append("gen_snippet{}".format(i))

